# LOGIN POST
usr = 'JOSELUISGOMEZGOMEZ'
pwd = 'josegmailgomez1'

b_url = 'https://www.serviciosenlinea.pjf.gob.mx/juicioenlinea/Home/Login'

body = {
    'UserName': usr,
    'UserPassword': pwd,
}
html = requests.post(b_url, body)

soup = bs4.BeautifulSoup(html.text)
print(soup)

In [1]:
import bs4
import requests
from time import sleep
from datetime import datetime

In [9]:
def scrap_circuitos(url, circuito):
    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.text)
    
    # Circuito
    c_name = soup.find(
        'input', {
            'name': 'CircuitoName',
        })
    c_name = c_name['value']    
    
    c_id = soup.find(
        'input', {
            'name': 'Circuito',
        })
    c_id = c_id['value']
    
    # Organismos
    select = soup.find(
        'select', {
            'name': 'Organismo',
        })
    options = select.findAll('option')

    # Object
    results = {
        'c_name': c_name,
        'c_id': c_id,
        'organismos': {}
    }
    if len(options) > 1:
        for o in options:
            o_id = o.get('value')
            o_txt = o.text

            data = {
                'org_id': o_id,
                'cir_id': c_id,
            }
            results['organismos'][o_id] = {
                o_txt : 'tipos_obj',
                #o_txt: scrap_tipo(data),
            }

        return True, results
    return False, results

In [10]:
# GET circuitos
def get_circuitos():
    cirs_ids = [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
        20, 30, 38, 39, 40, 41, 42, 43,
        44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 109
    ]
    circuitos = {}
    #for i in range(1, 1000):
    #for i in range(1, 2):
    for i in cirs_ids:
        b_url = f'https://www.dgepj.cjf.gob.mx/internet/expedientes/circuitos.asp?Cir={i}'
        flag, result = scrap_circuitos(b_url, i)
        if flag:
            circuitos[i] = result
    return circuitos

In [11]:
s = datetime.now()
circuitos = get_circuitos()

print(f'{datetime.now() - s} secs \n')

0:00:07.529126 secs 



In [15]:
print(circuitos.keys())
print()
print(circuitos[1]['organismos']['10'])
print()
print(circuitos[109]['organismos'].keys())
print(circuitos[109]['organismos']['1088'])

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 109])

{'Juzgado Primero de Distrito en Materia Administrativa en la Ciudad de México': 'tipos_obj'}

dict_keys(['125', '134', '2424', '1088', '1089'])
{'Tribunal Unitario del Trigésimo Segundo Circuito': 'tipos_obj'}


In [ ]:
{
    'Juzgado Primero de Distrito en Materia Administrativa en la Ciudad de México': {
        '1': 'Amparo indirecto',
        '2': 'Causa penal',
        '46': 'Concursos Mercantiles',
        '71': 'Denuncia por Incumplimiento de la Declaratoria General de Inconstitucionalidad',
        '67': 'Ejecución de penas', 
        '58': 'Extinción de Dominio', 
        '68': 'Juicio Oral Mercantil', 
        '18': 'Medidas precautorias', 
        '19': 'Procedimientos de extradición', 
        '4': 'Procesos civiles o administrativos'
    }
}

In [22]:
def scrap_tipo(data):
    b_url = 'https://www.dgepj.cjf.gob.mx/internet/expedientes/ExpedienteyTipo.asp'
    body = {
        'Organismo': data['org_id'],
        'Buscar': 'Buscar',
        'Circuito': data['cir_id'],
    }
    html = requests.post(b_url, body)
    soup = bs4.BeautifulSoup(html.text)
    
    # Tipos
    select = soup.find(
        'select', {
            'name': 'TipoAsunto',
        })
    options = select.findAll('option')
    
    # Object
    results = {} 
    if len(options) > 1:
        for o in options:
            results[o.get('value')] = o.text

        return results

In [23]:
#POST tipo
data = {
    'org_id': 109,
    'cir_id': 134,
}
tipos = scrap_tipo(data)

In [24]:
print(tipos.keys())

print(tipos['10'])

dict_keys(['29', '10', '11', '12', '17', '13', '23', '25', '15', '20', '26', '24', '14', '16'])
Amparo directo


In [5]:
# Acuerdos GET
def get_acuerdos(data):
    t_ast = data['t_ast']
    id_org = data['id_org']
    n_exp = data['n_exp']

    b_url = 'https://www.dgepj.cjf.gob.mx/siseinternet/reportes/vercaptura.aspx?'
    form = f'tipoasunto={t_ast}&organismo={id_org}&expediente={n_exp}&tipoprocedimiento=0'

    html = requests.get(b_url + form)
    soup = bs4.BeautifulSoup(html.text)

    select = soup.find(
            'table', {
                'id': 'grvAcuerdos',
            })
    tabla = select.findAll('tr')
    titles = tabla[0].findAll('th')

    acuerdos = []
    for a in tabla[1:]:
        ac_data = a.findAll('td')
        acuerdo = {}
        for v in range(len(titles[:-1])):
            acuerdo[titles[v].text] = ac_data[v].text
        acuerdos.append(acuerdo)

    for ac in acuerdos:
        ac['No.'] = ac['No.'].replace('\n', '')
    
    return acuerdos

In [8]:
data = {
    't_ast': 71,
    'id_org': 10,
    'n_exp': '1/2020',
}
print(get_acuerdos(data)[0]['Resumen'])

Acepta competencia y admite    
Notifíquese a las partes de la siguiente forma:   
   
Parte   
Tipo de notificación   
   
Denunciante   
Lista   
   
Autoridad denunciada   
Oficio   
   



In [ ]:
[
    {
        'No.': '1',
        'Fecha del Auto': '03-01-2020',
        'Tipo Cuaderno': 'Denuncia por Incumplimiento de la Declaratoria General de Inconstitucionalidad',
        'Fecha de publicación': '06-01-2020',
        'Resumen': 'Acepta competencia y admite    \nNotifíquese a las partes de la siguiente forma:   \n   \nParte   \nTipo de notificación   \n   \nDenunciante   \nLista   \n   \nAutoridad denunciada   \nOficio   \n   \n'
    },
    {
        'No.': '2', 'Fecha del Auto': '10-01-2020', 'Tipo Cuaderno': 'Denuncia por Incumplimiento de la Declaratoria General de Inconstitucionalidad', 'Fecha de publicación': '13-01-2020', 'Resumen': 'Autoridades, se les tiene realizando las manifestaciones que a su parte corresponden con relación a la denuncia.   \nDomicilio y delegados de las autoridades denunciadas. Medios electrónicos  \n  \nNotifíquese a las partes de la siguiente forma:  \n  \nParte   ...'},
    {
        'No.': '3', 'Fecha del Auto': '17-01-2020', 'Tipo Cuaderno': 'Denuncia por Incumplimiento de la Declaratoria General de Inconstitucionalidad', 'Fecha de publicación': '20-01-2020', 'Resumen': 'Por lo expuesto y fundado, se resuelve: \nPrimero. Es improcedente la denuncia por incumplimiento...  \nSegundo. Es infundada la denuncia por incumplimiento de la declaratoria general de inconstitucionalidad. \nNotifíquese a las partes de la siguiente forma: ...'},
    {
        'No.': '4', 'Fecha del Auto': '24-02-2020', 'Tipo Cuaderno': 'Denuncia por Incumplimiento de la Declaratoria General de Inconstitucionalidad', 'Fecha de publicación': '25-02-2020', 'Resumen': 'Juzgado Segundo de Distrito del Centro Auxiliar de la Primera Región, con residencia en la Ciudad de México; hágase de su conocimiento que este órgano jurisdiccional se avocó al conocimiento del asunto. Causa estado la resolución de dieciséis de enero de  ...'
    }
]
